In [ ]:
import os
import yaml
from random import randrange
import shutil

: 

In [ ]:
images_train_dir = os.path.join("", 'images', 'train')
images_val_dir = os.path.join("", 'images', 'val')
labels_train_dir = os.path.join("", 'labels', 'train')
labels_val_dir = os.path.join("", 'labels', 'val')

os.makedirs(images_train_dir, exist_ok=True)
os.makedirs(images_val_dir, exist_ok=True)
os.makedirs(labels_train_dir, exist_ok=True)
os.makedirs(labels_val_dir, exist_ok=True)

dataset_dir = os.path.join("", "dataset")
os.makedirs(dataset_dir, exist_ok=True)

In [ ]:
def create_yolo_training_config(path, product_specific=False, product_type_id=None, names=None):
    # Ensure the path exists
    if not os.path.exists(path):
        os.makedirs(path)

    # Create the folder structure
    images_train_dir = os.path.join('images', 'train')
    images_val_dir = os.path.join('images', 'val')
    labels_train_dir = os.path.join('labels', 'train')
    labels_val_dir = os.path.join('labels', 'val')

    os.makedirs(images_train_dir, exist_ok=True)
    os.makedirs(images_val_dir, exist_ok=True)
    os.makedirs(labels_train_dir, exist_ok=True)
    os.makedirs(labels_val_dir, exist_ok=True)

    dataset_dir = os.path.join("", "dataset")
    os.makedirs(dataset_dir, exist_ok=True)

    (labels_names_val, labels_names_train) = split_data()
    #update train and val sets
    update_label(labels_names_val, labels_val_dir, product_type_id)
    update_label(labels_names_train, labels_train_dir, product_type_id)

    # Create the YAML content
    yaml_content = {
        'train': os.path.join(path, 'images', 'train'),
        'val': os.path.join(path, 'images', 'val'),
        'labels': {
            'train': labels_train_dir,
            'val': labels_val_dir
        },
        'nc': len(names) if names else 0,
        'names': names if names else []
    }
    if product_specific and product_type_id is not None:
        yaml_content['product_type_id'] = product_type_id

    # Write the YAML file
    yaml_path = os.path.join(path, 'data.yaml')

    with open(yaml_path, 'w') as yaml_file:
        yaml.dump(yaml_content, yaml_file, default_flow_style=False)
    print(f'YOLO training configuration created at {yaml_path}')
    print(f'Folder structure created under {path}')

In [ ]:
def count_jpg_images(folder_path, format = ".jpg"):
    jpg_count = 0
    txt_count = 0
    if format == ".jpg":
      for file_name in os.listdir(folder_path):
        if file_name.endswith('.jpg'):
            jpg_count += 1
      return jpg_count
    elif format == ".txt":
      for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            txt_count += 1
      return txt_count

In [ ]:
def get_jpg_image_names(folder_path, format = ".jpg"):
    if format == ".jpg":
      jpg_images = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.jpg')]
      return jpg_images
    else:
      txt_images = [file_name for file_name in os.listdir(folder_path) if file_name.endswith('.txt')]
      return txt_images

In [168]:
def copy_image(src_folder, dest_folder, image_name):
    # Ensure the destination folder exists
    os.makedirs(dest_folder, exist_ok=True)

    # Define the full file paths
    src_path = os.path.join(src_folder, image_name)
    dest_path = os.path.join(dest_folder, image_name)

    # Copy the image
    shutil.copy(src_path, dest_path)
    #print(f'Copied {image_name} to {dest_folder}')

In [169]:
def copy_info_to_new_file(src_folder, dest_folder, file_name):
    # Ensure the destination folder exists
    os.makedirs(dest_folder, exist_ok=True)

    # Define the full file paths
    src_path = os.path.join(src_folder, file_name)
    dest_path = os.path.join(dest_folder, file_name)

    # Copy the file
    shutil.copy(src_path, dest_path)
    #print(f'Copied {file_name} to {dest_folder}')

In [170]:
def split_data():
  dataset_dir = 'dataset'

  image_train_dir = 'images/train'
  image_val_dir = 'images/val'

  labels_train_dir = 'labels/train'
  labels_val_dir = 'labels/val'

  number_of_images = count_jpg_images(dataset_dir)

  image_names = get_jpg_image_names(dataset_dir)
  image_names_index = {}

  label_names = get_jpg_image_names(dataset_dir, ".txt")
  label_names_index = {}

  labels_names_train = []
  labels_names_val = []

  #add image and label names into the dictionary
  for i in range(len(image_names)):
    image_names_index[image_names[i]] = randrange(10)
    label_names_index[label_names[i]] = image_names_index[image_names[i]]

  for i in range(number_of_images):
    if(list(image_names_index.values())[i] == 9 or list(image_names_index.values())[i] == 0):
      copy_image(dataset_dir, image_val_dir, list(image_names_index.keys())[i])
      copy_file(dataset_dir, labels_val_dir, list(label_names_index.keys())[i])
      labels_names_val.append(list(label_names_index.keys())[i])
    else:
      copy_image(dataset_dir, image_train_dir, list(image_names_index.keys())[i])
      copy_file(dataset_dir, labels_train_dir, list(label_names_index.keys())[i])
      labels_names_train.append(list(label_names_index.keys())[i])

  return (labels_names_val, labels_names_train)

In [171]:
"""label_names = get_jpg_image_names(dataset_dir, ".txt")

with open(dataset_dir + "/" + label_names[0], "r") as fileref:
  for i in fileref:
    print(i.split()[2:])

st = " ".join(i.split()[2:])"""

'label_names = get_jpg_image_names(dataset_dir, ".txt")\n\nwith open(dataset_dir + "/" + label_names[0], "r") as fileref:\n  for i in fileref:\n    print(i.split()[2:])\n\nst = " ".join(i.split()[2:])'

In [ ]:

def update_label(label_names, path, product_type_id=0):
  st = ""
  for index in range(len(label_names)):
    with open(path + "/" + label_names[index], "r") as fileref: # reads txt file 
      for i in fileref: # looks at each line
        if product_type_id == int(i.split()[1]):
          st = " ".join(i.split()[2:]) # takes everything but first two rows in a line


    with open(path + "/" + label_names[index], 'w') as file: # writes the txt file
      file.write(st) 

In [173]:
#update_label(labels_names_val, labels_val_dir)
products = {"Toilet":0, "Bathtub":1, "Sink":2, "Vanity":3, "Faucet":4}
print(products["Sink"])

2


In [174]:
products = {"Toilet":0, "Bathtub":1, "Sink":2, "Vanity":3, "Faucet":4}

# Example usage
create_yolo_training_config("path/to/directory", product_specific=True, product_type_id=products["Sink"], names=["Toilet1", "Toilet2", "Toilet3", "Toilet4"])



YOLO training configuration created at path/to/directory/data.yaml
Folder structure created under path/to/directory
